In [ ]:
!pip install fredapi --user

In [ ]:
import os

In [ ]:
os.chdir('C:/Users/shery/OneDrive/Desktop/Anaconda/fred')

In [6]:
import fredapi as fa
import pandas as pd
from config import fred as settings
fred = fa.Fred(settings['api_key'])
import datetime
from datetime import timedelta


In [20]:
#list of quarterly, monthly, & daily series
q_series = [('gdp','GDP')] 
m_series = [('capacity_manuf','MCUMFN'),('cpi','CPIAUCSL'),('unemp','UNRATE'),('leadin_ind','USALOLITONOSTSAM')]
d_series = [('fed_funds_rate','DFF'),('tbill_3','DTB3'),('tbill_6','DTB6')]
all_series = [item for sublist in [q_series, m_series, d_series] for item in sublist]


['GDP', '1947-01-01', '2023-04-01']

In [66]:
#Get the starting and end of series
date_df =[]

for varname, var in all_series:
    x= fred.search(var).observation_start
    x= x[var].strftime("%Y-%m-%d")
    y= fred.search(var).observation_end
    y= y[var].strftime("%Y-%m-%d")
    #list=[varname,x,y]
    #date_df.append(list)
    date_df.append({'Variable': varname, 'Start': x, 'End': y})


date_df = pd.DataFrame(date_df)
date_df.sort_values(by='Start')

,Variable,Start,End
0,gdp,1947-01-01,2023-04-01
2,cpi,1947-01-01,2023-08-01
3,unemp,1948-01-01,2023-08-01
6,tbill_3,1954-01-04,2023-09-19
5,fed_funds_rate,1954-07-01,2023-09-19
7,tbill_6,1958-12-09,2023-09-19
4,leadin_ind,1960-01-01,2023-08-01
1,capacity_manuf,1972-01-01,2023-08-01


Get all the quarterly series

In [105]:
from datetime import timedelta
q_df =[]

for varname, var in q_series:
    x = fred.get_series(var)
    x.name = varname
    x = pd.DataFrame(x)
    q_df.append(x)

q_df = pd.concat(q_df, axis=1)
q_df.index = q_df.index - timedelta(days=1)
q_df['quarter'] = q_df.index.quarter
q_df['year'] = q_df.index.year
q_df['q_year'] = q_df['quarter'].astype(str) +'_'+ q_df['year'].astype(str)
q_df.set_index('q_year', inplace=True)

In [106]:
q_df.tail()

,gdp,quarter,year
q_year,,,
1_2022,25248.476,1,2022
2_2022,25723.941,2,2022
3_2022,26137.992,3,2022
4_2022,26529.774,4,2022
1_2023,26798.605,1,2023


Get all the monthly series

In [88]:
m_df =[]

for varname, var in m_series:
    x = fred.get_series(var)
    x.name = varname
    x = pd.DataFrame(x)
    m_df.append(x)

m_df = pd.concat(m_df, axis=1)
m_df['quarter'] = m_df.index.quarter
m_df['year'] = m_df.index.year
m_df['q_year'] = m_df['quarter'].astype(str) +'_'+ m_df['year'].astype(str)


In [89]:
m_df

,capacity_manuf,cpi,unemp,leadin_ind,quarter,year,q_year
1947-01-01,NaN,21.480,NaN,NaN,1,1947,1_1947
1947-02-01,NaN,21.620,NaN,NaN,1,1947,1_1947
1947-03-01,NaN,22.000,NaN,NaN,1,1947,1_1947
1947-04-01,NaN,22.000,NaN,NaN,2,1947,2_1947
1947-05-01,NaN,21.950,NaN,NaN,2,1947,2_1947
...,...,...,...,...,...,...,...
2023-04-01,78.4065,302.918,3.4,99.016121,2,2023,2_2023
2023-05-01,78.1317,303.294,3.7,99.086985,2,2023,2_2023
2023-06-01,77.6937,303.841,3.6,99.185852,2,2023,2_2023
2023-07-01,77.8633,304.348,3.5,99.293305,3,2023,3_2023


In [99]:
m_df_mean= m_df.groupby('q_year').mean()

In [101]:
m_df_mean.sort_values(by=['year','quarter'])
m_df_mean.sample()

,capacity_manuf,cpi,unemp,leadin_ind,quarter,year
q_year,,,,,,
3_1964,NaN,31.05,5.0,100.754607,3.0,1964.0


In [102]:
d_df =[]

for varname, var in d_series:
    x = fred.get_series(var)
    x.name = varname
    x = pd.DataFrame(x)
    d_df.append(x)

d_df = pd.concat(d_df, axis=1)
d_df['quarter'] = d_df.index.quarter
d_df['year'] = d_df.index.year
d_df['q_year'] = d_df['quarter'].astype(str) +'_'+ d_df['year'].astype(str)


In [104]:
d_df_mean= d_df.groupby('q_year').mean()
d_df_mean.sort_values(by=['year','quarter'])
d_df_mean.sample()
d_df_mean.tail()

,fed_funds_rate,tbill_3,tbill_6,quarter,year
q_year,,,,,
4_2018,2.220109,2.310820,2.441148,4.0,2018.0
4_2019,1.645652,1.576935,1.573387,4.0,2019.0
4_2020,0.088804,0.092903,0.102742,4.0,2020.0
4_2021,0.079457,0.054677,0.095161,4.0,2021.0
4_2022,3.653370,4.043770,4.405246,4.0,2022.0


In [128]:
#merge datasets
df0 = q_df.merge(m_df_mean, left_index=True, right_index=True, how='inner')
df = df0.merge(d_df_mean, left_index=True, right_index=True, how='inner')


In [129]:
#clean data
df.drop(columns=['quarter_x', 'year_x'], inplace=True)


In [130]:
df['quarter']= df['quarter'].astype(int)
df['year']= df['year'].astype(int)

In [131]:
## only include data starting 1972 when all the fields have data.
df = df[df['year']>=1972]

In [126]:
df.head()

,gdp,capacity_manuf,cpi,unemp,leadin_ind,quarter_y,year_y,fed_funds_rate,tbill_3,tbill_6,quarter,year
q_year,,,,,,,,,,,,
1_1972,1266.369,81.730000,41.333333,5.766667,101.770549,1.0,1972.0,3.549011,3.445397,3.814603,1,1972
2_1972,1290.566,82.769833,41.600000,5.700000,102.354293,2.0,1972.0,4.301429,3.770938,4.232344,2,1972
3_1972,1328.904,83.237233,41.933333,5.566667,102.803953,3.0,1972.0,4.741413,4.210952,4.715397,3,1972
4_1972,1377.490,85.668433,42.366667,5.366667,103.297792,4.0,1972.0,5.145543,4.862833,5.173000,4,1972


In [132]:
df.shape

(205, 12)

In [136]:
df.to_csv("data.csv")

In [134]:
os.path

<module 'ntpath' from 'C:\\Users\\shery\\anaconda3\\lib\\ntpath.py'>